In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
os.environ["CUDA_LAUNCH_BLOCKING"]= "1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["TORCH_USE_CUDA_DSA"] = "1"

import random
import warnings
import nest_asyncio

nest_asyncio.apply()
warnings.filterwarnings("ignore")

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = ""

In [5]:
import re
import uuid
from llama_index.llms.openai import OpenAI
from tqdm.notebook import tqdm
import json
from sklearn.model_selection import train_test_split
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import MetadataMode
import pandas as pd

data = pd.read_excel("../Raw_data/LegalQA_COLING2020/training_qc_data.xlsx")
data.head()

,query,corpus
0,Công chứng hợp đồng quyền sử dụng đất của hộ g...,Bộ luật số 91/2015/QH13\nĐiều 212: Sở hữu chun...
1,Thời gian và nội dung công bố danh sách người ...,Nghị định số 85/2013/NĐ-CP\nĐiều 23: Thời gian...
2,Vợ không thực hiện nghĩa vụ cấp dưỡng cho con ...,Luật số 52/2014/QH13\nĐiều 119: Người có quyền...
3,Gây ồn giữa đêm bị xử phạt vi phạm hành chính ...,Nghị định số 167/2013/NĐ-CP\nĐiều 6: Vi phạm q...
4,Chứng minh tài sản riêng trong thời kỳ hôn nhân?,Luật số 52/2014/QH13\nĐiều 43: Tài sản riêng c...


In [6]:
train_set_, val_set_ = train_test_split(data, test_size=0.1, random_state=42)
train_set = train_set_.reset_index(drop=True)
val_set = val_set_.reset_index(drop=True)
print(train_set.shape)
print(val_set.shape)

(3754, 2)
(418, 2)


In [7]:
val_set.head()

,query,corpus
0,Quyền của bên thuê nhà ở khi nhà ở là tài sản ...,Nghị định số 163/2006/NĐ-CP\nĐiều 24: Thế chấp...
1,Con mang quốc tịch trung quốc có mẹ mang quốc ...,Luật số 24/2008/QH12\nĐiều 19: Điều kiện được ...
2,Cơ quan có thẩm quyền thực hiện thủ tục về chứ...,Nghị định số 23/2015/NĐ-CP\nĐiều 5: Thẩm quyền...
3,Quy định của pháp luật về việc lấy lời khai củ...,Bộ luật số 101/2015/QH13\nĐiều 421: Lấy lời kh...
4,Có bắt buộc phải có lời chứng trong chứng thực...,Nghị định số 23/2015/NĐ-CP\nĐiều 12: Lời chứng...


In [8]:
def generate_answers(
    data,
    prompt_template=None,
    verbose=False,
):
    """
    Automatically generate hypothetical questions that could be answered with
    doc in the corpus.
    """
    llm = OpenAI(model='gpt-3.5-turbo-16k')

    prompt_template = prompt_template or """\
    Bạn là một luật sư chuyên nghiệp. 
    Hãy dựa vào thông tin về điều luật sau và trả lời câu hỏi một cách đầy đủ và chính xác.
    Câu trả lời phải bao gồm những thông tin cơ bản về ngữ cảnh: Khoản mấy, Điều mấy và văn bản luật nào
    ----------------------
    {context_str}
    ------------- ---------\
    Câu hỏi: {question}
    """

    queries = {}
    answers = {}
    corpuses = {}
    relevant_docs = {}
        
    for index, row in data.iterrows():
        if index % 50 == 0:
            print("Processing at index ", index)
        query = {}
        answer = {}
        corpus = {}
        relevant_doc = {}
        
        context_text = row["corpus"]
        question = row["query"]
        query_ = prompt_template.format(context_str=context_text, question = question)
        response = llm.complete(query_)

        result = str(response).strip()
         
        question_id = str(uuid.uuid4()) 
        node_id = str(uuid.uuid4())  
        query[question_id] = question
        answer[question_id] = result
        corpus[node_id] = context_text
        relevant_doc[question_id] = [node_id]
        
        queries.update(query)
        answers.update(answer)
        corpuses.update(corpus)
        relevant_docs.update(relevant_doc)
        
    return queries, relevant_docs, answers, corpuses

In [9]:
train_queries, train_relevant_docs, train_answers, train_corpus = generate_answers(train_set)

Processing at index  0
Processing at index  50
Processing at index  100
Processing at index  150
Processing at index  200
Processing at index  250
Processing at index  300
Processing at index  350
Processing at index  400
Processing at index  450
Processing at index  500
Processing at index  550
Processing at index  600
Processing at index  650
Processing at index  700
Processing at index  750
Processing at index  800
Processing at index  850
Processing at index  900
Processing at index  950
Processing at index  1000
Processing at index  1050
Processing at index  1100
Processing at index  1150
Processing at index  1200
Processing at index  1250
Processing at index  1300
Processing at index  1350
Processing at index  1400
Processing at index  1450
Processing at index  1500
Processing at index  1550
Processing at index  1600
Processing at index  1650
Processing at index  1700
Processing at index  1750
Processing at index  1800
Processing at index  1850
Processing at index  1900
Processin

In [10]:
TRAIN_DATASET_FPATH = '../data_rag/coling2020/train_dataset.json'
train_dataset = {
    'queries': train_queries,
    'answers': train_answers,
    'corpus': train_corpus,
    'relevant_docs': train_relevant_docs
}
with open(TRAIN_DATASET_FPATH, 'w+') as f:
    json.dump(train_dataset, f)

In [ ]:
import json

def remove_invalid_answers(dataset):
    # Danh sách các id không có giá trị
    invalid_ids = []
    for id_, answer in dataset['answers'].items():
        # Kiểm tra xem câu trả lời có giá trị không
        if not answer.strip():
            invalid_ids.append(id_)
        if "Không có thông tin cụ thể về việc này trong đoạn văn trên" in answer:
            invalid_ids.append(id_)
            print(id_)
    
    # Loại bỏ các id không có giá trị
    for id_ in invalid_ids:
        # Loại bỏ câu hỏi tương ứng
        if id_ in dataset['queries']:
            del dataset['queries'][id_]
        # Loại bỏ câu trả lời không có giá trị
        del dataset['answers'][id_]
        # Loại bỏ văn bản liên quan tương ứng
        if id_ in dataset['relevant_docs']:
            del dataset['relevant_docs'][id_]
    
    return dataset
# Sử dụng hàm để loại bỏ các câu trả lời không có giá trị
with open(TRAIN_DATASET_FPATH, 'r', encoding='utf-8') as json_file:
    train_dataset = json.load(json_file)

clean_train_dataset = remove_invalid_answers(train_dataset)
with open(TRAIN_DATASET_FPATH, 'w+') as f:
    json.dump(clean_train_dataset, f)